In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import pydantic
from pypdf import PdfReader


load_dotenv(override=True)
gemini_api_key = os.getenv("GOOGLE_API_KEY")
if gemini_api_key:
    print(f"Gemini API Key exists and begins {gemini_api_key[:2]}")
else:
    print("Gemini API Key does not exist")



In [ ]:
reader = PdfReader("Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(linkedin)

In [ ]:
summary = ""
for line in open("summary.txt"):
    summary += line

print(summary)

In [ ]:
name = "Vaibhavi"

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

system_prompt

In [ ]:
openai = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key=gemini_api_key)

Model Evaluation 

In [ ]:
from pydantic import BaseModel

class Evaluator(BaseModel):
    is_accepted: bool
    feedback: str


In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
def evaluate(reply, message, history) -> Evaluator:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluator)
    return response.choices[0].message.parsed

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_accepted:
        print("Passed evaluation - returning reply")
        return reply
    else:
        print("Failed evaluation - retrying - " + evaluation.feedback)
        return rerun(reply, message, history, evaluation.feedback)
        

In [ ]:
gr.ChatInterface(chat, type="messages").launch()